# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse    Shipments             Customer  \
0  Feb  2 2023 12:45PM  256527        19  ADV80011276  AdvaGen Pharma, Ltd   
1  Feb  2 2023 12:45PM  256527        19  ADV80011277  AdvaGen Pharma, Ltd   
2  Feb  2 2023 12:45PM  256527        19  ADV80011278  AdvaGen Pharma, Ltd   
3  Feb  2 2023 12:45PM  256527        19  ADV80011279  AdvaGen Pharma, Ltd   
4  Feb  2 2023 12:45PM  256527        19  ADV80011281  AdvaGen Pharma, Ltd   
5  Feb  2 2023 12:45PM  256527        19  ADV80011282  AdvaGen Pharma, Ltd   
6  Feb  2 2023 12:45PM  256527        19  ADV80011283  AdvaGen Pharma, Ltd   
7  Feb  2 2023 12:45PM  256527        19  ADV80011284  AdvaGen Pharma, Ltd   
8  Feb  2 2023 12:45PM  256527        19  ADV80011286  AdvaGen Pharma, Ltd   
9  Feb  2 2023 12:45PM  256527        19  ADV80011287  AdvaGen Pharma, Ltd   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
47  256536       Released          1
48  256537       Released          1
49  256538       Released          1
50  256539       Released          1
51  256540       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
256536                NaN        NaN       1.0
256537                NaN        NaN       1.0
256538                NaN        NaN       1.0
256539                NaN        NaN       1.0
256540                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256414                0.0        1.0       0.0
256418                3.0        3.0       5.0
256427                0.0        0.0       1.0
256440                0.0        0.0       6.0
256444                0.0        0.0      10.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
256414                  0          1         0
256418                  3          3         5
256427                  0          0         1
256440                  0          0         6
256444                  0          0        10

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               256414          0          1         0
1               256418          3          3         5
2               256427          0          0         1
3               256440          0          0         6
4               256444          0          0        10

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               256414                   1         
1               256418         3         3        5
2               256427                            1
3               256440                            6
4               256444                           10

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0   Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd
56  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.
57  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.
58  Feb  2 2023 12:39PM  256534        16  ACG North America LLC
59  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.
60  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.
61  Feb  2 2023 12:38PM  256536        19  VITABIOTICS  USA INC.
62  Feb  2 2023 12:37PM  256535        10       Methapharm, Inc.
67  Feb  2 2023 12:30PM  256531        21      NBTY Global, Inc.
68  Feb  2 2023 12:29PM  256529        21      NBTY Global, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd             
1  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.             
2  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.             
3  Feb  2 2023 12:39PM  256534        16  ACG North America LLC             
4  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.             
5  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.             
6  Feb  2 2023 12:38PM  256536        19  VITABIOTICS  USA INC.             
7  Feb  2 2023 12:37PM  256535        10       Methapharm, Inc.             
8  Feb  2 2023 12:30PM  256531        21      NBTY Global, Inc.             
9  Feb  2 2023 12:29PM  256529        21      NBTY Global, Inc.             

  Executing Released  
0                 56  
1                  1  
2                  1  
3                  1  
4                  1  
5                  1  
6                  1  
7                  5  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd       56   
1  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.        1   
2  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.        1   
3  Feb  2 2023 12:39PM  256534        16  ACG North America LLC        1   
4  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.        1   
5  Feb  2 2023 12:38PM  256537        21      NBTY Global, Inc.        1   
6  Feb  2 2023 12:38PM  256536        19  VITABIOTICS  USA INC.        1   
7  Feb  2 2023 12:37PM  256535        10       Methapharm, Inc.        5   
8  Feb  2 2023 12:30PM  256531        21      NBTY Global, Inc.        1   
9  Feb  2 2023 12:29PM  256529        21      NBTY Global, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd       56   
1  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.        1   
2  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.        1   
3  Feb  2 2023 12:39PM  256534        16  ACG North America LLC        1   
4  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.        1   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd      56.0   
1  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.       1.0   
2  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.       1.0   
3  Feb  2 2023 12:39PM  256534        16  ACG North America LLC       1.0   
4  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.       1.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Feb  2 2023 12:45PM  256527        19    AdvaGen Pharma, Ltd      56.0   
1  Feb  2 2023 12:41PM  256540        22      NBTY Global, Inc.       1.0   
2  Feb  2 2023 12:41PM  256539        22      NBTY Global, Inc.       1.0   
3  Feb  2 2023 12:39PM  256534        16  ACG North America LLC       1.0   
4  Feb  2 2023 12:39PM  256538        10       Methapharm, Inc.       1.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         4616768      68.0       19.0        3.0
12         1282416       3.0        2.0        0.0
15         1025878      25.0       15.0       19.0
16          256534       1.0        0.0        0.0
19         1538946      60.0        1.0        0.0
20          256519       1.0        0.0        0.0
21         2052031       8.0        0.0        0.0
22          769566       3.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  4616768      68.0       19.0        3.0
1        12  1282416       3.0        2.0        0.0
2        15  1025878      25.0       15.0       19.0
3        16   256534       1.0        0.0        0.0
4        19  1538946      60.0        1.0        0.0
5        20   256519       1.0        0.0        0.0
6        21  2052031       8.0        0.0        0.0
7        22   769566       3.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      68.0       19.0        3.0
1        12       3.0        2.0        0.0
2        15      25.0       15.0       19.0
3        16       1.0        0.0        0.0
4        19      60.0        1.0        0.0
5        20       1.0        0.0        0.0
6        21       8.0        0.0        0.0
7        22       3.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   68.0
1        12  Released    3.0
2        15  Released   25.0
3        16  Released    1.0
4        19  Released   60.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10   12    15   16    19   20   21   22
Status                                              
Completed   3.0  0.0  19.0  0.0   0.0  0.0  0.0  0.0
Executing  19.0  2.0  15.0  0.0   1.0  0.0  0.0  0.0
Released   68.0  3.0  25.0  1.0  60.0  1.0  8.0  3.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10   12    15   16    19   20   21   22
0          Completed   3.0  0.0  19.0  0.0   0.0  0.0  0.0  0.0
1          Executing  19.0  2.0  15.0  0.0   1.0  0.0  0.0  0.0
2           Released  68.0  3.0  25.0  1.0  60.0  1.0  8.0  3.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10   12    15   16    19   20   21   22
0  Completed   3.0  0.0  19.0  0.0   0.0  0.0  0.0  0.0
1  Executing  19.0  2.0  15.0  0.0   1.0  0.0  0.0  0.0
2   Released  68.0  3.0  25.0  1.0  60.0  1.0  8.0  3.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()